# Data Processing

This section of the notebook outlines a comprehensive pipeline to load, process, reshape, and prepare multi-modal time-series data for downstream analytical tasks. The pipeline integrates multiple datasets, merges them by `specimen_id` and `subject_id`, aligns them with temporal data from a clinical study (e.g., relative days to a booster vaccination), and ultimately constructs a 3D PyTorch tensor ready for machine learning workflows.

#### Overview of the Steps

1. **Data Loading:**
   - **Input Files:**
     - `TheData.xlsx`: Contains various sheets with biological measurements at different time points for subjects.
     - `training_pbmc_gene_expression_wide_tpm.tsv`: TPM (Transcripts Per Million) gene expression data aligned by `specimen_id`.
     - `genenames.tsv`: Contains the gene names or identifiers to filter the gene expression data.
   
   The code reads these files into pandas DataFrames for subsequent processing.

2. **Data Filtering and Alignment:**
   - **Gene Filtering:**  
     Using `genenames.tsv`, we create a list of target genes and filter the TPM dataset to retain only these genes and the `specimen_id` column.
   
   - **Temporal Data Mapping:**  
     From the `subject_specimen` sheet in `TheData.xlsx`, we extract timepoint information (`actual_day_relative_to_boost`) and create a mapping (`specimen_time_mapping`) to associate each `specimen_id` with its corresponding timepoint.

3. **Pivoting to Time-Series Format:**
   - Each relevant sheet (other than `subject_specimen`) is merged with the time mapping and pivoted into a time-series format. This results in DataFrames keyed by `subject_id` (rows) and timepoints (columns), with column names representing `feature_day<timepoint>`.

4. **Baseline Consolidation:**
   - Identification of baseline timepoints (e.g., those before day 0) is performed, and their values are consolidated into a single "baseline" (day0.0) feature by taking the median across all baseline measurements. This step is crucial when baseline values are spread across multiple negative timepoints.
   
   - A function `consolidate_baseline_features_handle_nan` is used to handle missing baseline data and ensure a consistent baseline reference point across subjects.

5. **Chunked Processing of Large DataFrames:**
   - Some DataFrames may be very large (tens of thousands of columns). The code includes a function `process_large_dataframe_in_chunks` to apply baseline consolidation in manageable pieces, avoiding memory issues.

6. **Combining Processed DataFrames:**
   - Once each individual DataFrame (gene expression, cellular frequency data, cytokine concentrations, antibody levels, etc.) has been processed, they are merged into a single `combined_dataframe` keyed by `subject_id`.
   
   - The final combined DataFrame contains a rich set of features (genes, proteins, cell frequencies) across multiple timepoints for each subject.

7. **Feature and Time Parsing:**
   - A helper function `parse_features_and_temporal_dimensions_refined` extracts the `feature_name` and `temporal_dimension` (timepoint) from column names. This parsing is necessary to later build structured tensors and ensure that each (feature, time) pair is identifiable.

8. **Constructing a 3D PyTorch Tensor:**
   - Using `create_3d_tensor_with_assertions`, we transform the combined DataFrame into a tensor of shape `[Number_of_Subjects, Number_of_Timepoints, Number_of_Features]`.
   - Subjects are sorted and indexed, timepoints are sorted and indexed, and features are sorted and indexed. Each entry in the tensor corresponds to a data value (e.g., gene expression at a given time and subject).

9. **Imputation via Cubic Spline Interpolation:**
   - Missing data (NaNs) are imputed using cubic spline interpolation along the time axis. This helps create a fully dense tensor suitable for many ML methods.
   
   - If a feature for a given subject is too sparsely sampled (e.g., fewer than two non-NaN points), that feature is skipped for that subject.

10. **Feature Filtering:**
    - Additional filtering steps identify features that have sufficient non-NaN coverage across a chosen set of timepoints.
    - Also, certain target features of interest are searched within the combined feature set to verify their presence and indices.

11. **Saving Results:**
    - The final imputed tensor and metadata (feature sets, temporal mappings, indices) are saved as pickle files for future use. This ensures that subsequent analyses can start from a well-structured, clean dataset.

#### Key Outputs:
- **`combined_dataframe`**: A single large table containing all subjects, features, and timepoints.
- **`Data_tensor_on_cpu`**: The raw tensor before imputation.
- **`imputed_tensor`**: The final tensor after cubic spline interpolation, ready for modeling.
- **`imputed_tensor_train_data.pkl`**: A serialized pickle file containing the imputed tensor and related metadata.
- **`training_data_Tensor_save.pkl`** and other pickle files: Intermediate and final data structures for debugging, backups, and downstream analyses.

This modularizes the entire data processing... The resulting clean, tensor format sets the stage for machine learning ...

In [1]:
import pandas as pd
import numpy as np
import os

file_1_path = 'harmonized_data/train/TheData.xlsx'
file_2_path = 'harmonized_data/train/training_pbmc_gene_expression_wide_tpm.tsv'
file_path = 'harmonized_data/genenames.tsv'
# Ensure the folder exists
os.makedirs('./tmp/',  exist_ok=True)

tpm_data = pd.read_csv(file_2_path, sep='\t')
genenames_df = pd.read_csv(file_path, sep='\t')
genenames_array = genenames_df.iloc[:, 0].to_numpy()
columns_to_keep = ['specimen_id'] + list(genenames_array)
tpm_data = tpm_data.loc[:, tpm_data.columns.intersection(columns_to_keep)]
excel_data = pd.ExcelFile(file_1_path)
sheet_dataframes = {sheet: excel_data.parse(sheet) for sheet in excel_data.sheet_names}
sheet_columns = {sheet: df.columns.tolist() for sheet, df in sheet_dataframes.items()}

subject_specimen_df = sheet_dataframes['subject_specimen']
if 'actual_day_relative_to_boost' in subject_specimen_df.columns:
    specimen_time_mapping = subject_specimen_df[['specimen_id', 'actual_day_relative_to_boost']]
    specimen_time_mapping = specimen_time_mapping.rename(columns={'actual_day_relative_to_boost': 'timepoint'})
else:
    raise ValueError("Timepoint data is missing from the 'subject_specimen' sheet.")

timeseries_dataframes = {}
for sheet_name, df in sheet_dataframes.items():
    if sheet_name != 'subject_specimen':
        df_with_time = pd.merge(df, specimen_time_mapping, on='specimen_id', how='left')
        time_series = df_with_time.pivot(index='specimen_id', columns='timepoint')
        timeseries_dataframes[sheet_name] = time_series

timeseries_shapes = {sheet: df.shape for sheet, df in timeseries_dataframes.items()}

if 'actual_day_relative_to_boost' in subject_specimen_df.columns:
    subject_specimen_mapping = subject_specimen_df[['specimen_id', 'subject_id', 'actual_day_relative_to_boost']]
else:
    subject_specimen_mapping = subject_specimen_df[['specimen_id', 'subject_id', 'planned_day_relative_to_boost']]
    subject_specimen_mapping = subject_specimen_mapping.rename(columns={'planned_day_relative_to_boost': 'actual_day_relative_to_boost'})

restructured_dataframes_corrected = {}
for sheet_name, df in timeseries_dataframes.items():
    df_reset = df.reset_index()
    df_reset.columns = [
        '_'.join(map(str, col)) if isinstance(col, tuple) else col for col in df_reset.columns
    ]
    if 'specimen_id_' in df_reset.columns:
        df_reset = df_reset.rename(columns={'specimen_id_': 'specimen_id'})
    df_with_mapping = pd.merge(df_reset, subject_specimen_mapping, on='specimen_id', how='left')
    df_pivoted = df_with_mapping.pivot_table(
        index='subject_id',
        columns=['actual_day_relative_to_boost'],
        values=[col for col in df_with_mapping.columns if col not in ['specimen_id', 'subject_id', 'actual_day_relative_to_boost']]
    )
    df_pivoted.columns = [
        f"{col[0]}_day{col[1]}" if isinstance(col, tuple) else col for col in df_pivoted.columns
    ]
    restructured_dataframes_corrected[sheet_name] = df_pivoted

t_cell_polarization_df = restructured_dataframes_corrected['t_cell_polarization']
t_cell_activation_df = restructured_dataframes_corrected['t_cell_activation']
plasma_cytokine_conc_O_df = restructured_dataframes_corrected['plasma_cytokine_conc_O']
plasma_cytokine_conc_L_df = restructured_dataframes_corrected['plasma_cytokine_conc_L']
plasma_antibody_levels_df = restructured_dataframes_corrected['plasma_antibody_levels']
pbmc_cell_frequency_df = restructured_dataframes_corrected['pbmc_cell_frequency']

available_baseline_timepoints = subject_specimen_mapping['actual_day_relative_to_boost'].unique()
baseline_timepoints_scanned = sorted([tp for tp in available_baseline_timepoints if tp in available_baseline_timepoints[available_baseline_timepoints<0]])
baseline_timepoints = baseline_timepoints_scanned

tpm_data_mapped = pd.merge(
    tpm_data,
    subject_specimen_mapping[['specimen_id', 'subject_id', 'actual_day_relative_to_boost']],
    on='specimen_id',
    how='left'
)

tpm_data_pivoted = tpm_data_mapped.pivot_table(
    index='subject_id',
    columns='actual_day_relative_to_boost',
    values=[col for col in tpm_data_mapped.columns if col not in ['specimen_id', 'subject_id', 'actual_day_relative_to_boost']]
)
tpm_data_pivoted.columns = [f"{col[0]}_day{col[1]}" for col in tpm_data_pivoted.columns]
tpm_data_final = tpm_data_pivoted

np.seterr(invalid="ignore")

def consolidate_baseline_features_handle_nan(df, baseline_timepoints):
    baseline_columns = [col for col in df.columns if any(f"day{tp}" in col for tp in baseline_timepoints)]
    non_baseline_columns = [col for col in df.columns if col not in baseline_columns]
    if not baseline_columns:
        return df
    baseline_values = df[baseline_columns].to_numpy()
    baseline_medians = np.nanmedian(baseline_values, axis=1)
    all_nan_rows = np.isnan(baseline_values).all(axis=1)
    baseline_medians[all_nan_rows] = np.nan
    baseline_feature_names = set(col.split("_day")[0] for col in baseline_columns)
    consolidated_baseline = pd.DataFrame(
        {f"{feature}_day0.0": baseline_medians for feature in baseline_feature_names},
        index=df.index
    )
    non_baseline_df = df[non_baseline_columns]
    consolidated_df = pd.concat([consolidated_baseline, non_baseline_df], axis=1)
    return consolidated_df

def process_large_dataframe_in_chunks(df, baseline_timepoints, chunk_size=50000):
    column_names = df.columns
    num_chunks = (len(column_names) // chunk_size) + 1
    result_dfs = []
    for i in range(num_chunks):
        start_idx = i * chunk_size
        end_idx = min(start_idx + chunk_size, len(column_names))
        chunk_columns = column_names[start_idx:end_idx]
        chunk_df = df[chunk_columns]
        processed_chunk = consolidate_baseline_features_handle_nan(chunk_df, baseline_timepoints)
        result_dfs.append(processed_chunk)
    return pd.concat(result_dfs, axis=1)

dataframes = {
    "tpm_data_final": tpm_data_final,
    "t_cell_polarization_df": t_cell_polarization_df,
    "t_cell_activation_df": t_cell_activation_df,
    "plasma_cytokine_conc_O_df": plasma_cytokine_conc_O_df,
    "plasma_cytokine_conc_L_df": plasma_cytokine_conc_L_df,
    "plasma_antibody_levels_df": plasma_antibody_levels_df,
    "pbmc_cell_frequency_df": pbmc_cell_frequency_df
}

processed_dataframes = {}
processing_status = {}
for name, df in dataframes.items():
    try:
        if df.shape[1] > 100000:
            processed_dataframes[name] = process_large_dataframe_in_chunks(df, baseline_timepoints, chunk_size=50000)
        else:
            processed_dataframes[name] = consolidate_baseline_features_handle_nan(df, baseline_timepoints)
        processing_status[name] = "completed"
    except Exception as e:
        processing_status[name] = f"error: {e}"

tpm_data_final_processed = processed_dataframes.get("tpm_data_final", None)
t_cell_polarization_df_processed = processed_dataframes.get("t_cell_polarization_df", None)
t_cell_activation_df_processed = processed_dataframes.get("t_cell_activation_df", None)
plasma_cytokine_conc_O_df_processed = processed_dataframes.get("plasma_cytokine_conc_O_df", None)
plasma_cytokine_conc_L_df_processed = processed_dataframes.get("plasma_cytokine_conc_L_df", None)
plasma_antibody_levels_df_processed = processed_dataframes.get("plasma_antibody_levels_df", None)
pbmc_cell_frequency_df_processed = processed_dataframes.get("pbmc_cell_frequency_df", None)

dataframes_to_combine = [
    pbmc_cell_frequency_df_processed,
    plasma_antibody_levels_df_processed,
    plasma_cytokine_conc_L_df_processed,
    plasma_cytokine_conc_O_df_processed,
    t_cell_activation_df_processed,
    t_cell_polarization_df_processed,
    tpm_data_final_processed
]

combined_dataframe = None
for df in dataframes_to_combine:
    if df is not None:
        if combined_dataframe is None:
            combined_dataframe = df
        else:
            combined_dataframe = pd.merge(combined_dataframe, df, on="subject_id", how="outer")

combined_dataframe_shape = combined_dataframe.shape if combined_dataframe is not None else (0, 0)

def parse_features_and_temporal_dimensions_refined(df):
    temporal_columns = [col for col in df.columns if "_day" in col]
    parsed_data = []
    for col in temporal_columns:
        try:
            feature_name, temporal_info = col.rsplit("_day", 1)
            temporal_parts = temporal_info.split("_")
            temporal_dimension = int(float(temporal_parts[0]))
            if len(temporal_parts) > 1:
                suffix = "_" + temporal_parts[-1]
                feature_name += suffix
            feature_name_parts = feature_name.split("_")
            feature_name = "_".join(
                part for part in feature_name_parts if not part.replace(".", "").replace("-", "").isdigit()
            )
            parsed_data.append({
                "feature_name": feature_name,
                "temporal_dimension": temporal_dimension,
                "column_name": col
            })
        except Exception:
            continue
    return pd.DataFrame(parsed_data)

parsed_features_complex_df = parse_features_and_temporal_dimensions_refined(combined_dataframe)

import torch
import pandas as pd
import numpy as np
import pickle
from scipy.interpolate import CubicSpline
from tqdm import tqdm

unique_temporal_values = np.sort(parsed_features_complex_df["temporal_dimension"].unique())
temporal_mapping = {f"T{i}": (i, value) for i, value in enumerate(unique_temporal_values)}
unique_features = np.sort(parsed_features_complex_df["feature_name"].unique())
target_features = [
    "ENSG00000277632.1",
    "DMSO_P01579", "DMSO_Q16552", "DMSO_P05113", "PHA_P01579",
    "PHA_Q16552", "PHA_P05113", "PT_P01579", "PT_Q16552", "PT_P05113",
    "Monocytes", "IgG_PT"
]

FeaturesToIncludeIndices = [
    np.where(unique_features == feature)[0][0] if feature in unique_features else -1
    for feature in target_features
]

def create_3d_tensor_with_assertions(df, parsed_features_df, device="cpu"):
    unique_subjects = df.index.astype(int).unique()
    unique_temporal_dimensions = np.sort(parsed_features_df["temporal_dimension"].unique())
    unique_features_local = np.sort(parsed_features_df["feature_name"].unique())
    N, T, F = len(unique_subjects), len(unique_temporal_dimensions), len(unique_features_local)
    tensor = torch.full((N, T, F), float('nan'), dtype=torch.float, device=device)
    subject_to_idx = {subject: i for i, subject in enumerate(unique_subjects)}
    temporal_to_idx = {t: i for i, t in enumerate(unique_temporal_dimensions)}
    feature_to_idx = {feature: i for i, feature in enumerate(unique_features_local)}
    parsed_features_df["feature_idx"] = parsed_features_df["feature_name"].map(feature_to_idx)
    parsed_features_df["temporal_idx"] = parsed_features_df["temporal_dimension"].map(temporal_to_idx)
    valid_parsed_features = parsed_features_df[parsed_features_df["column_name"].isin(df.columns)]
    grouped = valid_parsed_features.groupby(["feature_idx", "temporal_idx"])
    for (feature_idx, temporal_idx), group in tqdm(grouped, desc="Processing Groups", total=len(grouped)):
        columns = group["column_name"].values
        valid_columns = [col for col in columns if col in df.columns]
        if not valid_columns:
            continue
        aggregated_values = df[valid_columns].min(axis=1, skipna=True).values
        subject_indices = torch.tensor(df.index.map(subject_to_idx).values, dtype=torch.long, device=device)
        tensor[subject_indices, temporal_idx, feature_idx] = torch.tensor(aggregated_values, dtype=torch.float, device=device)
    return tensor, unique_subjects, unique_temporal_dimensions, unique_features_local

device = "cuda" if torch.cuda.is_available() else "cpu"
tensor_3d_with_assertions, subjects, temporal_dims, features = create_3d_tensor_with_assertions(
    combined_dataframe, parsed_features_complex_df, device=device
)
Data_tensor_on_cpu = tensor_3d_with_assertions.cpu()

with open("./tmp/training_data_Tensor_save.pkl", "wb") as f:
    pickle.dump(Data_tensor_on_cpu, f)

def impute_with_cubic_interpolation_safe(tensor, time_dim=1):
    device = tensor.device
    tensor = tensor.cpu()
    imputed_tensor = tensor.clone()
    skipped_features = []
    for i in range(tensor.size(0)):
        for j in range(tensor.size(2)):
            data = tensor[i, :, j].numpy()
            time_indices = ~torch.isnan(tensor[i, :, j])
            times = torch.arange(tensor.size(1))
            if time_indices.sum() < 2:
                skipped_features.append(j)
                continue
            cubic_spline = CubicSpline(times[time_indices], data[time_indices], extrapolate=True)
            imputed_values = cubic_spline(times)
            imputed_tensor[i, :, j] = torch.tensor(imputed_values, dtype=torch.float32)
    skipped_features = list(set(skipped_features))
    return imputed_tensor.to(device), skipped_features

imputed_tensor, skipped_features = impute_with_cubic_interpolation_safe(Data_tensor_on_cpu, time_dim=1)

def get_non_nan_features_for_timepoints(tensor, timepoints):
    valid_features_set = set(range(tensor.size(2)))
    batch_size = tensor.size(0)
    threshold = batch_size/2
    for timepoint in timepoints:
        non_nan_counts = (~torch.isnan(tensor[:, timepoint, :])).sum(dim=0)
        valid_features_mask = non_nan_counts >= threshold
        valid_feature_indices = set(torch.nonzero(valid_features_mask, as_tuple=True)[0].tolist())
        valid_features_set &= valid_feature_indices
    return torch.tensor(sorted(valid_features_set))

timepoints_to_check = [0, 10, 16]
non_nan_features_indices = get_non_nan_features_for_timepoints(imputed_tensor, timepoints_to_check)

if isinstance(non_nan_features_indices, torch.Tensor):
    non_nan_features_indices_list = non_nan_features_indices.tolist()

combined_features = sorted(set(non_nan_features_indices_list + FeaturesToIncludeIndices))
unique_features_combined = unique_features[combined_features]

TargetedFeaturesToEvaluate_combined = [
    np.where(unique_features_combined == feature)[0][0] if feature in unique_features_combined else -1
    for feature in target_features
]

mapped_feature_pairs = {
    feature: (TargetedFeaturesToEvaluate_combined[i] if TargetedFeaturesToEvaluate_combined[i] != -1 else "Not Found")
    for i, feature in enumerate(target_features)
}


def count_features_with_non_nan_bounds_per_subject(tensor, bounds):
    counts_per_subject = {}
    for subject_id in range(tensor.size(0)):
        counts = {
            (start, end): (~torch.isnan(tensor[subject_id, start, :]) & ~torch.isnan(tensor[subject_id, end, :])).sum().item()
            for start, end in bounds
        }
        counts_per_subject[subject_id] = counts
    return counts_per_subject

bounds = [(0, 10), (0, 16), (0, 26)]
feature_counts_per_subject = count_features_with_non_nan_bounds_per_subject(imputed_tensor, bounds)

# print("Counts of features with non-NaN values at specified bounds per subject:")
# for subject_id, counts in feature_counts_per_subject.items():
#     print(f"Subject ID {subject_id}:")
#     for bound, count in counts.items():
#         print(f"  Bounds {bound}: {count} features")

min_threshold = 6000
subject_ids_with_low_features_count = [
    subject_id
    for subject_id, counts in feature_counts_per_subject.items()
    if all(count < min_threshold for count in counts.values())
]

# print(f"\nSubject IDs with all bounds having less than {min_threshold} features:")
# print(subject_ids_with_low_features_count)
print(f"Number of subjects: {len(subject_ids_with_low_features_count)}")


variables_to_save = {
    "imputed_tensor": imputed_tensor,
    "combined_features": combined_features,
    "unique_features": unique_features,
    "unique_features_combined": unique_features_combined,
    "subject_ids_with_low_features_count": subject_ids_with_low_features_count,
    "TargetedFeaturesToEvaluate_combined": TargetedFeaturesToEvaluate_combined,
    "unique_temporal_values": unique_temporal_values,
    "temporal_mapping": temporal_mapping
}

with open("./tmp/imputed_tensor_train_data.pkl", "wb") as f:
    pickle.dump(variables_to_save, f)



/common/ssbio/aosinuga2/mytorchC/lib/python3.12/site-packages/numpy/lib/nanfunctions.py:1217: RuntimeWarning: All-NaN slice encountered
  return function_base._ureduce(a, func=_nanmedian, keepdims=keepdims,
/common/ssbio/aosinuga2/mytorchC/lib/python3.12/site-packages/numpy/lib/nanfunctions.py:1217: RuntimeWarning: All-NaN slice encountered
  return function_base._ureduce(a, func=_nanmedian, keepdims=keepdims,
/common/ssbio/aosinuga2/mytorchC/lib/python3.12/site-packages/numpy/lib/nanfunctions.py:1217: RuntimeWarning: All-NaN slice encountered
  return function_base._ureduce(a, func=_nanmedian, keepdims=keepdims,
/common/ssbio/aosinuga2/mytorchC/lib/python3.12/site-packages/numpy/lib/nanfunctions.py:1217: RuntimeWarning: All-NaN slice encountered
  return function_base._ureduce(a, func=_nanmedian, keepdims=keepdims,
/common/ssbio/aosinuga2/mytorchC/lib/python3.12/site-packages/numpy/lib/nanfunctions.py:1217: RuntimeWarning: All-NaN slice encountered
  return function_base._ureduce(a, f

Counts of features with non-NaN values at specified bounds per subject:
Subject ID 0:
  Bounds (0, 10): 6634 features
  Bounds (0, 16): 6634 features
  Bounds (0, 26): 6634 features
Subject ID 1:
  Bounds (0, 10): 6634 features
  Bounds (0, 16): 6634 features
  Bounds (0, 26): 6634 features
Subject ID 2:
  Bounds (0, 10): 6684 features
  Bounds (0, 16): 6684 features
  Bounds (0, 26): 6684 features
Subject ID 3:
  Bounds (0, 10): 6634 features
  Bounds (0, 16): 6634 features
  Bounds (0, 26): 6634 features
Subject ID 4:
  Bounds (0, 10): 6684 features
  Bounds (0, 16): 6684 features
  Bounds (0, 26): 6684 features
Subject ID 5:
  Bounds (0, 10): 31 features
  Bounds (0, 16): 31 features
  Bounds (0, 26): 31 features
Subject ID 6:
  Bounds (0, 10): 6634 features
  Bounds (0, 16): 6634 features
  Bounds (0, 26): 6634 features
Subject ID 7:
  Bounds (0, 10): 6634 features
  Bounds (0, 16): 6634 features
  Bounds (0, 26): 6634 features
Subject ID 8:
  Bounds (0, 10): 6684 features
  Bounds

# Challenge Dataset

In [2]:
import pandas as pd
import numpy as np
import os

file_1_path = 'harmonized_data/challenge/TheData_challenge.xlsx'
file_2_path = 'harmonized_data/challenge/challenge_pbmc_gene_expression_wide_tpm.tsv'
file_path = 'harmonized_data/genenames.tsv'
# Ensure the folder exists
os.makedirs('./tmp/',  exist_ok=True)

tpm_data = pd.read_csv(file_2_path, sep='\t')
genenames_df = pd.read_csv(file_path, sep='\t')
genenames_array = genenames_df.iloc[:, 0].to_numpy()
columns_to_keep = ['specimen_id'] + list(genenames_array)
tpm_data = tpm_data.loc[:, tpm_data.columns.intersection(columns_to_keep)]
excel_data = pd.ExcelFile(file_1_path)
sheet_dataframes = {sheet: excel_data.parse(sheet) for sheet in excel_data.sheet_names}
sheet_columns = {sheet: df.columns.tolist() for sheet, df in sheet_dataframes.items()}

subject_specimen_df = sheet_dataframes['subject_specimen']
if 'actual_day_relative_to_boost' in subject_specimen_df.columns:
    specimen_time_mapping = subject_specimen_df[['specimen_id', 'actual_day_relative_to_boost']]
    specimen_time_mapping = specimen_time_mapping.rename(columns={'actual_day_relative_to_boost': 'timepoint'})
else:
    raise ValueError("Timepoint data is missing from the 'subject_specimen' sheet.")

timeseries_dataframes = {}
for sheet_name, df in sheet_dataframes.items():
    if sheet_name != 'subject_specimen':
        df_with_time = pd.merge(df, specimen_time_mapping, on='specimen_id', how='left')
        time_series = df_with_time.pivot(index='specimen_id', columns='timepoint')
        timeseries_dataframes[sheet_name] = time_series

timeseries_shapes = {sheet: df.shape for sheet, df in timeseries_dataframes.items()}

if 'actual_day_relative_to_boost' in subject_specimen_df.columns:
    subject_specimen_mapping = subject_specimen_df[['specimen_id', 'subject_id', 'actual_day_relative_to_boost']]
else:
    subject_specimen_mapping = subject_specimen_df[['specimen_id', 'subject_id', 'planned_day_relative_to_boost']]
    subject_specimen_mapping = subject_specimen_mapping.rename(columns={'planned_day_relative_to_boost': 'actual_day_relative_to_boost'})

restructured_dataframes_corrected = {}
for sheet_name, df in timeseries_dataframes.items():
    df_reset = df.reset_index()
    df_reset.columns = [
        '_'.join(map(str, col)) if isinstance(col, tuple) else col for col in df_reset.columns
    ]
    if 'specimen_id_' in df_reset.columns:
        df_reset = df_reset.rename(columns={'specimen_id_': 'specimen_id'})
    df_with_mapping = pd.merge(df_reset, subject_specimen_mapping, on='specimen_id', how='left')
    df_pivoted = df_with_mapping.pivot_table(
        index='subject_id',
        columns=['actual_day_relative_to_boost'],
        values=[col for col in df_with_mapping.columns if col not in ['specimen_id', 'subject_id', 'actual_day_relative_to_boost']]
    )
    df_pivoted.columns = [
        f"{col[0]}_day{col[1]}" if isinstance(col, tuple) else col for col in df_pivoted.columns
    ]
    restructured_dataframes_corrected[sheet_name] = df_pivoted

t_cell_polarization_df = restructured_dataframes_corrected['t_cell_polarization']
t_cell_activation_df = restructured_dataframes_corrected['t_cell_activation']
plasma_cytokine_conc_O_df = restructured_dataframes_corrected['plasma_cytokine_conc_O']
plasma_cytokine_conc_L_df = restructured_dataframes_corrected['plasma_cytokine_conc_L']
plasma_antibody_levels_df = restructured_dataframes_corrected['plasma_antibody_levels']
pbmc_cell_frequency_df = restructured_dataframes_corrected['pbmc_cell_frequency']

available_baseline_timepoints = subject_specimen_mapping['actual_day_relative_to_boost'].unique()
baseline_timepoints_scanned = sorted([tp for tp in available_baseline_timepoints if tp in available_baseline_timepoints[available_baseline_timepoints<0]])
baseline_timepoints = baseline_timepoints_scanned

tpm_data_mapped = pd.merge(
    tpm_data,
    subject_specimen_mapping[['specimen_id', 'subject_id', 'actual_day_relative_to_boost']],
    on='specimen_id',
    how='left'
)

tpm_data_pivoted = tpm_data_mapped.pivot_table(
    index='subject_id',
    columns='actual_day_relative_to_boost',
    values=[col for col in tpm_data_mapped.columns if col not in ['specimen_id', 'subject_id', 'actual_day_relative_to_boost']]
)
tpm_data_pivoted.columns = [f"{col[0]}_day{col[1]}" for col in tpm_data_pivoted.columns]
tpm_data_final = tpm_data_pivoted

np.seterr(invalid="ignore")

def consolidate_baseline_features_handle_nan(df, baseline_timepoints):
    baseline_columns = [col for col in df.columns if any(f"day{tp}" in col for tp in baseline_timepoints)]
    non_baseline_columns = [col for col in df.columns if col not in baseline_columns]
    if not baseline_columns:
        return df
    baseline_values = df[baseline_columns].to_numpy()
    baseline_medians = np.nanmedian(baseline_values, axis=1)
    all_nan_rows = np.isnan(baseline_values).all(axis=1)
    baseline_medians[all_nan_rows] = np.nan
    baseline_feature_names = set(col.split("_day")[0] for col in baseline_columns)
    consolidated_baseline = pd.DataFrame(
        {f"{feature}_day0.0": baseline_medians for feature in baseline_feature_names},
        index=df.index
    )
    non_baseline_df = df[non_baseline_columns]
    consolidated_df = pd.concat([consolidated_baseline, non_baseline_df], axis=1)
    return consolidated_df

def process_large_dataframe_in_chunks(df, baseline_timepoints, chunk_size=50000):
    column_names = df.columns
    num_chunks = (len(column_names) // chunk_size) + 1
    result_dfs = []
    for i in range(num_chunks):
        start_idx = i * chunk_size
        end_idx = min(start_idx + chunk_size, len(column_names))
        chunk_columns = column_names[start_idx:end_idx]
        chunk_df = df[chunk_columns]
        processed_chunk = consolidate_baseline_features_handle_nan(chunk_df, baseline_timepoints)
        result_dfs.append(processed_chunk)
    return pd.concat(result_dfs, axis=1)

dataframes = {
    "tpm_data_final": tpm_data_final,
    "t_cell_polarization_df": t_cell_polarization_df,
    "t_cell_activation_df": t_cell_activation_df,
    "plasma_cytokine_conc_O_df": plasma_cytokine_conc_O_df,
    "plasma_cytokine_conc_L_df": plasma_cytokine_conc_L_df,
    "plasma_antibody_levels_df": plasma_antibody_levels_df,
    "pbmc_cell_frequency_df": pbmc_cell_frequency_df
}

processed_dataframes = {}
processing_status = {}
for name, df in dataframes.items():
    try:
        if df.shape[1] > 100000:
            processed_dataframes[name] = process_large_dataframe_in_chunks(df, baseline_timepoints, chunk_size=50000)
        else:
            processed_dataframes[name] = consolidate_baseline_features_handle_nan(df, baseline_timepoints)
        processing_status[name] = "completed"
    except Exception as e:
        processing_status[name] = f"error: {e}"

tpm_data_final_processed = processed_dataframes.get("tpm_data_final", None)
t_cell_polarization_df_processed = processed_dataframes.get("t_cell_polarization_df", None)
t_cell_activation_df_processed = processed_dataframes.get("t_cell_activation_df", None)
plasma_cytokine_conc_O_df_processed = processed_dataframes.get("plasma_cytokine_conc_O_df", None)
plasma_cytokine_conc_L_df_processed = processed_dataframes.get("plasma_cytokine_conc_L_df", None)
plasma_antibody_levels_df_processed = processed_dataframes.get("plasma_antibody_levels_df", None)
pbmc_cell_frequency_df_processed = processed_dataframes.get("pbmc_cell_frequency_df", None)

dataframes_to_combine = [
    pbmc_cell_frequency_df_processed,
    plasma_antibody_levels_df_processed,
    plasma_cytokine_conc_L_df_processed,
    plasma_cytokine_conc_O_df_processed,
    t_cell_activation_df_processed,
    t_cell_polarization_df_processed,
    tpm_data_final_processed
]

combined_dataframe = None
for df in dataframes_to_combine:
    if df is not None:
        if combined_dataframe is None:
            combined_dataframe = df
        else:
            combined_dataframe = pd.merge(combined_dataframe, df, on="subject_id", how="outer")

combined_dataframe_shape = combined_dataframe.shape if combined_dataframe is not None else (0, 0)

def parse_features_and_temporal_dimensions_refined(df):
    temporal_columns = [col for col in df.columns if "_day" in col]
    parsed_data = []
    for col in temporal_columns:
        try:
            feature_name, temporal_info = col.rsplit("_day", 1)
            temporal_parts = temporal_info.split("_")
            temporal_dimension = int(float(temporal_parts[0]))
            if len(temporal_parts) > 1:
                suffix = "_" + temporal_parts[-1]
                feature_name += suffix
            feature_name_parts = feature_name.split("_")
            feature_name = "_".join(
                part for part in feature_name_parts if not part.replace(".", "").replace("-", "").isdigit()
            )
            parsed_data.append({
                "feature_name": feature_name,
                "temporal_dimension": temporal_dimension,
                "column_name": col
            })
        except Exception:
            continue
    return pd.DataFrame(parsed_data)

parsed_features_complex_df = parse_features_and_temporal_dimensions_refined(combined_dataframe)

import torch
import pandas as pd
import numpy as np
import pickle
from scipy.interpolate import CubicSpline
from tqdm import tqdm

# unique_temporal_values = np.sort(parsed_features_complex_df["temporal_dimension"].unique())
# temporal_mapping = {f"T{i}": (i, value) for i, value in enumerate(unique_temporal_values)}
# unique_features = np.sort(parsed_features_complex_df["feature_name"].unique())
# target_features = [
#     "ENSG00000277632.1",
#     "DMSO_P01579", "DMSO_Q16552", "DMSO_P05113", "PHA_P01579",
#     "PHA_Q16552", "PHA_P05113", "PT_P01579", "PT_Q16552", "PT_P05113",
#     "Monocytes", "IgG_PT"
# ]

# FeaturesToIncludeIndices = [
#     np.where(unique_features == feature)[0][0] if feature in unique_features else -1
#     for feature in target_features
# ]

def create_3d_tensor_with_assertions(df, parsed_features_df, device="cpu"):
    unique_subjects = df.index.astype(int).unique()
    unique_temporal_dimensions = np.sort(parsed_features_df["temporal_dimension"].unique())
    unique_features_local = np.sort(parsed_features_df["feature_name"].unique())
    N, T, F = len(unique_subjects), len(unique_temporal_dimensions), len(unique_features_local)
    tensor = torch.full((N, T, F), float('nan'), dtype=torch.float, device=device)
    subject_to_idx = {subject: i for i, subject in enumerate(unique_subjects)}
    temporal_to_idx = {t: i for i, t in enumerate(unique_temporal_dimensions)}
    feature_to_idx = {feature: i for i, feature in enumerate(unique_features_local)}
    parsed_features_df["feature_idx"] = parsed_features_df["feature_name"].map(feature_to_idx)
    parsed_features_df["temporal_idx"] = parsed_features_df["temporal_dimension"].map(temporal_to_idx)
    valid_parsed_features = parsed_features_df[parsed_features_df["column_name"].isin(df.columns)]
    grouped = valid_parsed_features.groupby(["feature_idx", "temporal_idx"])
    for (feature_idx, temporal_idx), group in tqdm(grouped, desc="Processing Groups", total=len(grouped)):
        columns = group["column_name"].values
        valid_columns = [col for col in columns if col in df.columns]
        if not valid_columns:
            continue
        aggregated_values = df[valid_columns].min(axis=1, skipna=True).values
        subject_indices = torch.tensor(df.index.map(subject_to_idx).values, dtype=torch.long, device=device)
        tensor[subject_indices, temporal_idx, feature_idx] = torch.tensor(aggregated_values, dtype=torch.float, device=device)
    return tensor, unique_subjects, unique_temporal_dimensions, unique_features_local

# device = "cuda" if torch.cuda.is_available() else "cpu"
# tensor_3d_with_assertions, subjects, temporal_dims, features = create_3d_tensor_with_assertions(
#     combined_dataframe, parsed_features_complex_df, device=device
# )
# Data_tensor_on_cpu = tensor_3d_with_assertions.cpu()

# with open("./tmp/challenge_data_Tensor_save.pkl", "wb") as f:
#     pickle.dump(Data_tensor_on_cpu, f)

# def impute_with_cubic_interpolation_safe(tensor, time_dim=1):
#     device = tensor.device
#     tensor = tensor.cpu()
#     imputed_tensor = tensor.clone()
#     skipped_features = []
#     for i in range(tensor.size(0)):
#         for j in range(tensor.size(2)):
#             data = tensor[i, :, j].numpy()
#             time_indices = ~torch.isnan(tensor[i, :, j])
#             times = torch.arange(tensor.size(1))
#             if time_indices.sum() < 2:
#                 skipped_features.append(j)
#                 continue
#             cubic_spline = CubicSpline(times[time_indices], data[time_indices], extrapolate=True)
#             imputed_values = cubic_spline(times)
#             imputed_tensor[i, :, j] = torch.tensor(imputed_values, dtype=torch.float32)
#     skipped_features = list(set(skipped_features))
#     return imputed_tensor.to(device), skipped_features

# imputed_tensor, skipped_features = impute_with_cubic_interpolation_safe(Data_tensor_on_cpu, time_dim=1)

import pickle
pickle_filepath = "./tmp/imputed_tensor_train_data.pkl"
# Load the variables back
with open(pickle_filepath, "rb") as f:
    loaded_variables = pickle.load(f)
# Verify loaded variables
combined_features_train = loaded_variables["combined_features"]
unique_features_train = loaded_variables["unique_features_combined"]
TargetedFeaturesToEvaluate_combined_train = loaded_variables["TargetedFeaturesToEvaluate_combined"]


unique_features = np.sort(parsed_features_complex_df["feature_name"].unique())
unique_features#.shape


# Find indices of unique_features_train in unique_features
unique_features_IntrainSetIndices = [np.where(unique_features == feature)[0][0] for feature in unique_features_train if feature in unique_features]
unique_features_use = unique_features[unique_features_IntrainSetIndices]
len(unique_features_IntrainSetIndices)

# List of features to find
target_features = [
    "ENSG00000277632.1",
    "DMSO_P01579", "DMSO_Q16552", "DMSO_P05113", "PHA_P01579",
    "PHA_Q16552", "PHA_P05113", "PT_P01579", "PT_Q16552", "PT_P05113",
    "Monocytes", "IgG_PT"
]

# Find indices of the target features in the unique_features array
FeaturesToIncludeIndices = [
    np.where(unique_features_use == feature)[0][0] if feature in unique_features_use else -1
    for feature in target_features
]

# Display the results
FeaturesToIncludeIndices

# Apply the function
device = "cuda" if torch.cuda.is_available() else "cpu"
tensor_3d_with_assertions, subjects, temporal_dims, features = create_3d_tensor_with_assertions(
    combined_dataframe, parsed_features_complex_df, device=device)
tensor_3d_with_assertions = tensor_3d_with_assertions[:, :, unique_features_IntrainSetIndices]
features = features[unique_features_IntrainSetIndices]
# Output the shape of the resulting tensor
print(f"Final tensor shape: {tensor_3d_with_assertions.shape}")



# Data_tensor_on_cpu = tensor_3d_no_chunking_vectorized.cpu()
Data_tensor_on_cpu = tensor_3d_with_assertions.cpu()
Data_tensor_on_cpu.shape

variables_to_save = {
    "challenge_tensor": Data_tensor_on_cpu,
    "features": features,
    "subject_ids": subjects,
}
with open("./tmp/tensor_challenge_data.pkl", "wb") as f:
    pickle.dump(variables_to_save, f)



/common/ssbio/aosinuga2/mytorchC/lib/python3.12/site-packages/torch/storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(io.BytesIO(b))
Processing

Final tensor shape: torch.Size([54, 1, 6693])


# Model Training

This section of the notebook takes previously processed and imputed data and proceeds through a series of steps to fit machine learning models and generate predictions for downstream tasks.

#### Overview of the Steps

1. **Data Loading and Filtering:**
   - Loads a previously saved, fully processed and imputed dataset (`imputed_tensor_train_data.pkl`).
   - Extracts the `imputed_tensor`, `combined_features`, and `subject_ids_with_low_features_count`.
   - Filters out subjects with insufficient feature coverage and selects only the `combined_features` of interest.  
   - The result is a filtered and cleaned PyTorch tensor `X_imputed_filtered` that is free of NaNs (replaced with zeros).

2. **Data Normalization:**
   - Splits the filtered data into an input slice `X_input` (the first timepoint) and target slices `Y_target` (subsequent timepoints).
   - A normalization step is performed across all features and timepoints combined, ensuring that each feature has zero mean and unit variance.  
   - The normalization parameters (mean and std) are stored for future use, such as de-normalization of predictions.

3. **Model Training Using XGBoost:**
   - Each timepoint of interest is modeled separately, allowing for flexible training strategies and comparisons across timepoints.
   - Flattened and normalized input features `X_normalized` at T0 serve as predictors, and the normalized values at each target timepoint `Y_normalized[:, t, :]` serve as targets.
   - The code uses XGBoost with GPU acceleration (if available) for efficient model training.  
   - Models are trained on a train-test split, and performance is measured using RMSE and Adjusted R² metrics.

4. **Model Evaluation:**
   - For each modeled timepoint, the RMSE and Adjusted R² metrics are computed and printed. This provides insight into how well the model captures the variability of the data and how well it generalizes.

5. **Model and Parameters Saving:**
   - Trained XGBoost models, evaluation metrics, and parameters are saved into a single pickle file for future use, ensuring reproducibility and easy model loading without retraining.

6. **Challenge Data Loading and Prediction:**
   - A "challenge_tensor" (i.e., a dataset provided for a predictive challenge) is loaded from another pickle file.
   - The challenge dataset is normalized using the previously obtained normalization parameters, ensuring consistent scaling with the training data.
   
   - Predictions for selected features at various timepoints are generated using the trained models.
   - A special case (`timepoint = -1`) directly extracts baseline features rather than producing a model prediction.

7. **De-normalizing Predictions:**
   - The predicted values, originally in normalized space, are de-normalized back into the original scale of the features.
   - This allows for interpreting model outputs in terms of the raw biological or clinical units measured originally.

8. **Ranking and Fold-Change Computations:**
   - Tasks defined by the challenge are addressed by:
     - Ranking subjects based on a single feature at a certain timepoint.
     - Computing fold-changes between two timepoints for a feature and ranking subjects accordingly.
     - Computing ratios of features (e.g., Th1/Th2) and ranking subjects based on these ratios.
   
   - All these computed rankings and ratio-based metrics are compiled into a final DataFrame, providing a consolidated summary of model-driven insights.

9. **Saving Results:**
   - The final rankings and fold-change results are saved to a CSV file, which can be submitted to the challenge or used for further analysis.

#### Key Outputs:
- **`imputed_tensor_train_data.pkl` and related input files:**  
  Previously prepared data and associated variables.
  
- **`model_param_data.pkl` (or similarly named file):**  
  Stores trained XGBoost models, their metrics, and parameters for reproducibility and downstream evaluation.
  
- **`task_rankings.csv`:**  
  Final computed rankings of subjects for each defined task, based on model predictions of selected features at given timepoints or fold-changes.



In [3]:
import pickle
import torch
import numpy as np
import pandas as pd
import cupy as cp
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import root_mean_squared_error

# Load processed data and variables
pickle_filepath = "./tmp/imputed_tensor_train_data.pkl"
with open(pickle_filepath, "rb") as f:
    loaded_variables = pickle.load(f)

imputed_tensor = loaded_variables["imputed_tensor"]
combined_features = loaded_variables["combined_features"]
subject_ids_with_low_features_count = loaded_variables["subject_ids_with_low_features_count"]
TargetedFeaturesToEvaluate_combined = loaded_variables["TargetedFeaturesToEvaluate_combined"]

X_imputed = imputed_tensor[:, :, combined_features]
all_subject_indices = torch.arange(X_imputed.size(0))
valid_subject_indices = [idx for idx in all_subject_indices if idx not in subject_ids_with_low_features_count]
X_imputed_filtered = X_imputed[valid_subject_indices]
X_imputed_filtered = torch.nan_to_num(X_imputed_filtered, nan=0.0)

X_input = X_imputed_filtered[:, 0, :].unsqueeze(1)
Y_target = X_imputed_filtered[:, 1:, :]

def normalize_3d(X, Y):
    combined = np.concatenate([X, Y], axis=1)
    feature_mean = np.mean(combined, axis=(0, 1), keepdims=True)
    feature_std = np.std(combined, axis=(0, 1), keepdims=True)
    normalized = (combined - feature_mean) / (feature_std + 1e-8)
    X_normalized = normalized[:, :1, :]
    Y_normalized = normalized[:, 1:, :]
    normalization_params = {"mean": feature_mean, "std": feature_std}
    return X_normalized, Y_normalized, normalization_params

X_normalized, Y_normalized, normalization_params = normalize_3d(X_input.numpy(), Y_target.numpy())

def adjusted_r2_from_rmse(rmse, y_true, n_features):
    n = len(y_true)
    variance_y = np.var(y_true)
    ss_total = variance_y * n
    ss_residual = (rmse ** 2) * n
    r2 = 1 - (ss_residual / ss_total)
    adjusted_r2 = 1 - ((1 - r2) * (n - 1) / (n - n_features - 1))
    return adjusted_r2

def flatten_Y(Y):
    num_samples, seq_len, output_dim = Y.shape
    return Y.reshape(num_samples, seq_len * output_dim)

timepoints = [0, 2, 9, 15]
models = {}
metrics = {}

seed = 42
np.random.seed(seed)

for t in timepoints:
    print(f"Training model for time point {t}...")
    Y_t = Y_normalized[:, t, :]
    X_train, X_test, Y_train, Y_test = train_test_split(
        X_normalized.squeeze(1),
        Y_t,
        test_size=0.2,
        random_state=seed
    )
    X_train = cp.array(X_train)
    X_test = cp.array(X_test)
    Y_train = cp.array(Y_train)
    Y_test = cp.array(Y_test)

    params = {
        # Param_1
        'n_estimators': 1272,
        'max_depth': 100,
        'max_leaves': 0,
        'learning_rate': 0.0753493322967014,
        'max_delta_step': 4,
        'lambda': 4,
        'subsample': 1,
        'objective': 'reg:squarederror',
        'early_stopping_rounds': 2,
        'n_jobs': -1,
        'verbosity': 2,
        'tree_method': 'hist',
        'random_state': 42,
        'eval_metric': 'rmse',
        'device': 'cuda' if torch.cuda.is_available() else 'cpu',

        # # Param_2
        # 'n_estimators': 318000,
        # 'max_depth': 500,
        # 'max_leaves': 0,
        # 'learning_rate': 0.0753493322967014,
        # 'max_delta_step': 4,
        # 'alpha': 1.230710427888436,
        # 'subsample': 1,
        # 'objective': 'reg:squarederror',
        # 'early_stopping_rounds': 1,
        # 'n_jobs': -1,
        # 'verbosity': 2,
        # 'tree_method': 'hist',
        # 'random_state': 42,
        # 'eval_metric': 'rmse',
        # 'device': 'cuda' if torch.cuda.is_available() else 'cpu',

        # # Param_3
        # 'n_estimators': 318,
        # 'max_depth': 50,
        # 'max_leaves': 0,
        # 'learning_rate': 0.0753493322967014,
        # 'max_delta_step': 4,
        # 'alpha': 1.230710427888436,
        # 'subsample': 1,
        # 'objective': 'reg:squarederror',
        # 'early_stopping_rounds': 2,
        # 'n_jobs': -1,
        # 'verbosity': 2,
        # 'tree_method': 'hist',
        # 'random_state': seed,
        # 'eval_metric': "rmse",
        # 'device': 'cuda' if torch.cuda.is_available() else 'cpu',
    }

    model = xgb.XGBRegressor(**params)
    model.fit(X_train, Y_train, eval_set=[(X_test, Y_test)], verbose=True)
    models[t] = model

    Y_test_pred = model.predict(X_test)
    test_rmse = root_mean_squared_error(Y_test.get(), Y_test_pred)
    adjusted_r2 = adjusted_r2_from_rmse(test_rmse, Y_test.get(), X_train.shape[1])
    metrics[t] = {'rmse': test_rmse, 'adjusted_r2': adjusted_r2}
# Print summary of metrics
for t, metric in metrics.items():
    print(f"Time Point {t}: Test RMSE = {metric['rmse']:.4f}, Adjusted R^2 = {metric['adjusted_r2']:.4f}")

save_data = {
    "models": models,
    "metrics": metrics,
    "params": params
}

save_file = "model_param_data_1.pkl"
# save_file = "model_param_data_2.pkl"
# save_file = "model_param_data_3.pkl"

with open(save_file, "wb") as f:
    pickle.dump(save_data, f)



/common/ssbio/aosinuga2/mytorchC/lib/python3.12/site-packages/torch/storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(io.BytesIO(b))


[0]	validation_0-rmse:0.13140
[1]	validation_0-rmse:0.12281
[2]	validation_0-rmse:0.11490
[3]	validation_0-rmse:0.10759
[4]	validation_0-rmse:0.10087
[5]	validation_0-rmse:0.09466
[6]	validation_0-rmse:0.08896
[7]	validation_0-rmse:0.08370
[8]	validation_0-rmse:0.07887
[9]	validation_0-rmse:0.07445
[10]	validation_0-rmse:0.07042
[11]	validation_0-rmse:0.06675
[12]	validation_0-rmse:0.06341
[13]	validation_0-rmse:0.06037
[14]	validation_0-rmse:0.05761
[15]	validation_0-rmse:0.05516
[16]	validation_0-rmse:0.05292
[17]	validation_0-rmse:0.05090
[18]	validation_0-rmse:0.04909
[19]	validation_0-rmse:0.04750
[20]	validation_0-rmse:0.04606
[21]	validation_0-rmse:0.04476
[22]	validation_0-rmse:0.04360
[23]	validation_0-rmse:0.04261
[24]	validation_0-rmse:0.04170
[25]	validation_0-rmse:0.04088
[26]	validation_0-rmse:0.04020
[27]	validation_0-rmse:0.03956
[28]	validation_0-rmse:0.03900
[29]	validation_0-rmse:0.03854
[30]	validation_0-rmse:0.03810
[31]	validation_0-rmse:0.03772
[32]	validation_0-

# Inference on Challenge Data

In [5]:

pickle_filepath = "./tmp/tensor_challenge_data.pkl"
with open(pickle_filepath, "rb") as f:
    loaded_variables = pickle.load(f)

challenge_tensor = loaded_variables["challenge_tensor"]
challenge_features = loaded_variables["features"]
challenge_subjects = loaded_variables["subject_ids"]

def denormalize_3d(data, params):
    mean = params["mean"]
    std = params["std"]
    return data * std + mean

def normalize_new_3d(X, params):
    mean = params["mean"]
    std = params["std"]
    normalized = (X - mean) / (std + 1e-8)
    return normalized

def predict_and_save_features_with_last_timepoint(X_normalized, selected_features, models, timepoints, subjects):
    results = []
    for t in timepoints:
        if t == -1:
            for feature_idx in selected_features:
                feature_values = X_normalized[:, feature_idx]
                for sample_idx, value in enumerate(feature_values):
                    results.append({
                        "Timepoint": t,
                        "Feature_Index": feature_idx,
                        "Sample_Index": subjects[sample_idx],
                        "Predicted_Value": value.cpu().numpy() if torch.is_tensor(value) else value
                    })
        else:
            model = models[t]
            Y_pred = model.predict(X_normalized)
            for feature_idx in selected_features:
                feature_values = Y_pred[:, feature_idx]
                for sample_idx, value in enumerate(feature_values):
                    results.append({
                        "Timepoint": t,
                        "Feature_Index": feature_idx,
                        "Sample_Index": subjects[sample_idx],
                        "Predicted_Value": value
                    })
    return pd.DataFrame(results)

X_normalized_new = normalize_new_3d(challenge_tensor, normalization_params).squeeze(1)
selected_features = [6643, 6645, 6607, 6679, 6680]
timepoints = [0, 2, 9, 15, -1]
predicted_features_df = predict_and_save_features_with_last_timepoint(X_normalized_new, selected_features, models, timepoints, challenge_subjects)

def denormalize_predictions(predictions_df, normalization_params):
    predictions_df["De_Normalized_Value"] = predictions_df.apply(
        lambda row: row["Predicted_Value"] * normalization_params["std"][0, 0, row["Feature_Index"]] +
                    normalization_params["mean"][0, 0, row["Feature_Index"]], axis=1
    )
    return predictions_df

predicted_features_df = denormalize_predictions(predicted_features_df, normalization_params)

tasks = {
    "1.1": {"feature": 6643, "timepoint": 9, "type": "rank"},
    "1.2": {"feature": 6643, "timepoint_1": 9, "timepoint_0": -1, "type": "fold_change"},
    "2.1": {"feature": 6645, "timepoint": 0, "type": "rank"},
    "2.2": {"feature": 6645, "timepoint_1": 0, "timepoint_0": -1, "type": "fold_change"},
    "3.1": {"feature": 6607, "timepoint": 2, "type": "rank"},
    "3.2": {"feature": 6607, "timepoint_1": 2, "timepoint_0": -1, "type": "fold_change"},
    "4.1": {"Th1": 6679, "Th2": 6680, "timepoint": 15, "type": "ratio"},
}

results = {"SubjectID": challenge_subjects}

for task, details in tasks.items():
    if details["type"] == "rank":
        feature_values = predicted_features_df[
            (predicted_features_df["Feature_Index"] == details["feature"]) &
            (predicted_features_df["Timepoint"] == details["timepoint"])
        ].sort_values(by="Predicted_Value", ascending=False)
        results[task] = feature_values["Predicted_Value"].rank(ascending=False).values
    elif details["type"] == "fold_change":
        timepoint_1_values = predicted_features_df[
            (predicted_features_df["Feature_Index"] == details["feature"]) &
            (predicted_features_df["Timepoint"] == details["timepoint_1"])
        ]["Predicted_Value"].values
        timepoint_0_values = predicted_features_df[
            (predicted_features_df["Feature_Index"] == details["feature"]) &
            (predicted_features_df["Timepoint"] == details["timepoint_0"])
        ]["Predicted_Value"].values
        fold_changes = timepoint_1_values / timepoint_0_values
        results[task] = pd.Series(fold_changes).rank(ascending=False).values
    elif details["type"] == "ratio":
        Th1_values = predicted_features_df[
            (predicted_features_df["Feature_Index"] == details["Th1"]) &
            (predicted_features_df["Timepoint"] == details["timepoint"])
        ]["Predicted_Value"].values
        Th2_values = predicted_features_df[
            (predicted_features_df["Feature_Index"] == details["Th2"]) &
            (predicted_features_df["Timepoint"] == details["timepoint"])
        ]["Predicted_Value"].values
        ratios = Th1_values / Th2_values
        results[task] = pd.Series(ratios).rank(ascending=False).values

final_results_df = pd.DataFrame(results)
output_filepath = "tasks_rankings_1.csv"
# output_filepath = "tasks_rankings_2.csv"
# output_filepath = "tasks_rankings_3.csv"

final_results_df.to_csv(output_filepath, index=False)

/common/ssbio/aosinuga2/mytorchC/lib/python3.12/site-packages/torch/storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(io.BytesIO(b))
